If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8001/tree?token=collect-street-view```  

This notebook describes a pipeline to download panoramas and their corresponding depth maps from Google Street View.  
It takes an input ```.csv``` describing a list of EPSG:4326 latitude/longitude pairs (as a list of length 2), each with an associated primary key, and attempts to save the panorama as a ```.jpg``` and a 3D cartesian point cloud derived from the depth map as a ```.csv``` in the target directory.  
A ```.csv``` indicating primary keys with found panoramas (which correspond to the filenames), and the true latitude, longitude, and elevation is also saved.

In [1]:
target_dir = "data"
input_latlons_fp = "shunfu-coordinates.csv"
pano_dir = "shunfu-panos"
point_cloud_dir = "shunfu-point-clouds"
log_filename = "shunfu-id.csv"

In [2]:
import aiocsv
import aiofiles
from aiohttp import ClientSession, ClientTimeout
from streetlevel import streetview

import asyncio
import csv
import json
import math
import pandas as pd
from pathlib import Path
import os

In [3]:
def depthmap_to_xyz(depthmap: list[list[float]],
                    xrange: tuple[float, float] = (-1.0, 1.0), yrange: tuple[float, float] = (-1.0, 1.0),
                    heading: float = 0,
                    rmin: float = 0.0, rmax: float = math.inf) -> list[list[float]]:
    pi = math.pi
    sin = math.sin
    cos = math.cos
    output = []
    width = len(depthmap[0])
    height = len(depthmap)
    x0 = xrange[0]
    dx = xrange[1] - x0
    y0 = yrange[0]
    dy = yrange[1] - y0
    h = -heading
    for i in range(height):
        for j in range(width):
            r = depthmap[i][j]
            if r < rmin or r > rmax:
                continue
            xnorm = ((j + 0.5) / width) * dx + x0
            ynorm = ((i + 0.5) / height) * dy + y0
            theta = -pi * xnorm
            phi = -pi / 2 * ynorm
            cartesian = [[-r * sin(h + theta) * cos(phi)],
                         [r * cos(h + theta) * cos(phi)],
                         [r * sin(phi)]]
            output.append([cartesian[0][0], cartesian[1][0], cartesian[2][0]])
    return output

async def find_pano_id(session, lat, lon, radius=50):
    try:
        return await streetview.find_panorama_async(lat, lon, session, radius=radius)
    except Exception as e:
        print(f"find_pano_id, ({lat}, {lon}): {e}")
        return None

async def find_pano_depth(session, panoid):
    try:
        return await streetview.find_panorama_by_id_async(panoid, session, download_depth=True)
    except Exception as e:
        print(f"find_pano_depth, {panoid}: {e}")
        return None
    
async def find_pano_full(session, lat, lon, radius=50):
    base_pano = await find_pano_id(session, lat, lon, radius)
    if base_pano:
        return await find_pano_depth(session, base_pano.id)
    else:
        return None

async def save_pano_point_cloud(target_dir, pano_dir, point_cloud_dir, id, lat, lon, session, csv_writer):
    pano = await find_pano_full(session, lat, lon)
    if pano and pano.depth and pano.heading and pano.elevation:
        try:
            image = await streetview.get_panorama_async(pano, session, zoom=0)
            point_cloud = depthmap_to_xyz(pano.depth.data[:,::-1], heading=pano.heading, rmin=1.1, rmax=60)
            with open(os.path.join(target_dir, pano_dir, f"{id}.jpg"), 'w') as fp:
                image.save(fp)
            with open(os.path.join(target_dir, point_cloud_dir, f"{id}.csv"), 'w') as fp:
                point_cloud_writer = csv.writer(fp)
                point_cloud_writer.writerows(point_cloud)
            await csv_writer.writerow([id, pano.lat, pano.lon, pano.elevation])
        except Exception as e:
            print(f"save_pano_point_cloud, {pano.id}: {e}")

async def main(target_dir, pano_dir, point_cloud_dir, ids, lats, lons, log_filename):
    Path(os.path.join(target_dir, pano_dir)).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(target_dir, point_cloud_dir)).mkdir(parents=True, exist_ok=True)
    async with ClientSession(timeout=ClientTimeout(total=None), raise_for_status=True, trust_env=True) as session:
        async with aiofiles.open(os.path.join(target_dir, log_filename), 'w+') as fp:
            csv_writer = aiocsv.AsyncWriter(fp)
            await csv_writer.writerow(["id", "lat", "lon", "elevation"])
            tasks = []
            for i in range(len(ids)):
                tasks.append(save_pano_point_cloud(target_dir, pano_dir, point_cloud_dir, ids[i], lats[i], lons[i], session, csv_writer))
            await asyncio.gather(*tasks)

In [4]:
df = pd.read_csv(os.path.join(target_dir, input_latlons_fp))
df.columns = ["id", "lat", "lon"]
df = df.drop_duplicates(subset=["id"])
ids = df["id"].to_list()
lats = df["lat"].to_list()
lons = df["lon"].to_list()

await main(target_dir, pano_dir, point_cloud_dir, ids, lats, lons, log_filename)

find_pano_depth, vJmzqRVKJZ4stQHZ5Fb7OQ: Server disconnected
find_pano_depth, 7Ji36yajtdBj6ppSSxy2eA: Server disconnected
find_pano_depth, pBDqiazSlv3qH5iBm40a6w: Server disconnected
find_pano_depth, NY5gqIJVDGDdncze8QRUmg: Server disconnected
find_pano_depth, OblbY2DkqdHMT1Zx3gSEvw: Server disconnected
find_pano_depth, c5YO2Pg0gv8larBFWgnJVg: Server disconnected
save_pano_point_cloud, l8maHM_Nzu_ZmGZChywzqg: Server disconnected
save_pano_point_cloud, POxGcpYS3ve6pq_T2uhoRA: Server disconnected
save_pano_point_cloud, T39Mp0rlgwKZ9CxfefG0Eg: Server disconnected
find_pano_depth, JvlgoEtlUHdh8ePQ-nZ99g: Cannot connect to host www.google.com:443 ssl:default [None]
find_pano_depth, xtTQvAiJuM8r9G3INAbyMA: Cannot connect to host www.google.com:443 ssl:default [None]
find_pano_depth, ZfhJ4vmHaHIdy-wwv3wsuw: Cannot connect to host www.google.com:443 ssl:default [None]
find_pano_depth, vAuEd6HKVdPbZSp3qtdMLg: Cannot connect to host www.google.com:443 ssl:default [None]
find_pano_depth, pRCiy4HO